In [1]:
import numpy as np
import pandas as pd
from scipy.stats import beta, cauchy, gamma
import time  # Pour suivre le temps d'exécution

In [3]:
def prior(n, hyper):
    X_1 = beta.rvs(hyper[0], hyper[1] + hyper[2], size=n)
    Temp = beta.rvs(hyper[1], hyper[2], size=n)
    X_2 = Temp * (1 - X_1)
    R_1 = X_1 - X_2
    R_2 = 2 * (X_1 + X_2) - 1
    return np.column_stack((R_1, R_2))

def simu(tht1, tht2, sigm, n):
    A = np.random.normal(0, sigm, size=n + 30)
    U = A[2:(n + 2)]
    V = A[1:(n + 1)]
    W = A[0:n]
    return U + tht1 * V + tht2 * W

def distautcor(Z, X):
    p = len(Z)
    a2 = np.sum(Z[2:p] * Z[0:(p - 2)])
    b2 = np.sum(X[2:p] * X[0:(p - 2)])
    a1 = np.sum(Z[1:p] * Z[0:(p - 1)])
    b1 = np.sum(X[1:p] * X[0:(p - 1)])
    return np.sqrt((a2 - b2)**2 + (a1 - b1)**2)


In [41]:
def ABCbas(X, N, sigm, hyper):
    res = prior(N, hyper)
    dist = np.apply_along_axis(lambda x: distautcor(X - np.mean(X), simu(x[0], x[1], sigm, len(X))), 1, res)
    return res[np.argmin(dist), :]

def statdir(U):
    q = (U[:, 1] + 2 * U[:, 0] + 1) / 4
    q = np.where(q != 0, q, np.finfo(float).eps)  # Remplacer les zéros par une très petite valeur
    
    return np.array([np.sum(np.log(q)), np.sum(np.log(np.abs(q - U[:, 0])))])
    
def ABChaut(Q1, Q2, L):
    Q = np.column_stack((Q1, Q2))
    hyper = np.random.exponential(size=(L, 3))
    dist = np.apply_along_axis(lambda x: np.sum((statdir(prior(len(Q1), x)) - statdir(Q))**2), 1, hyper)
    return hyper[np.argmin(dist), :]

def ABCsigm(Dat, j, parh1, parh2, parsigm1, parsigm2, N):
    dist = np.zeros(N)
    n = Dat.shape[0]
    w = np.arange(0, n, 3)
    si = 1 / gamma.rvs(parsigm1, scale=parsigm2, size=N)
    for i in range(N):
        dist[i] = np.abs(np.var(simu(parh1[j], parh2[j], si[i], n)[w]) - np.var(Dat[w, j]))
    return si[np.argmin(dist)]

def ABCsigmhaut(sigm, Q):
    dist = np.zeros(Q)
    n = len(sigm)
    pri = np.abs(cauchy.rvs(size=2*Q)).reshape((Q, 2))
    for i in range(Q):
        h = pri[i]
        
        # Générer une valeur pour h[0] et h[1]
        while True:
            try:
                y = 1 / gamma.rvs(h[0], scale=h[1], size=n)
                break
            except OverflowError:
                # Si une erreur de dépassement de capacité se produit, ajustez les paramètres h[0] et h[1]
                h = np.abs(cauchy.rvs(size=2))
        
        dist[i] = np.abs(np.sum(np.log(y)) - np.sum(np.log(sigm))) + np.abs(np.sum(y) - np.sum(sigm))
    return pri[np.argmin(dist), :]


# Simule une chaîne selon ABCGibbs,
# renvoie une liste de matrices : hyperparamètres, premier paramètre, second, sigma, hypersigma
# pour les paramètres et sigma, les colonnes correspondent aux séries temporelles.

X: C'est le dataset, les données sur lesquelles nous effectuons les analyses et les simulations.
N: Nombre de simulations ou d'échantillons à générer dans les fonctions ABC pour déduire chaque paramètre
# M simulations pour les hyperparamètres, P simulations pour sigma
# P2 simulations pour l'hypersigma, Npts nombre de points désirés.
sigm: C'est la variance utilisée dans la simulation du modèle AR.
hyper: Les paramètres hyperparamétriques du modèle.
res: Les résultats de la fonction.
dist: Les distances calculées entre les simulations et les données réelles.
U: Les données générées par la fonction prior, utilisées dans les étapes de calcul.
q: La quantité calculée pour les statistiques suffisantes dans la fonction statdir.
Q1 et Q2: Vecteurs des premier et deuxième paramètres respectivement, utilisés dans la fonction ABChaut.
L: Nombre de simulations pour l'inférence des hyperparamètres dans ABChaut.
Dat: L'ensemble de données complet utilisé dans ABCsigm.
j: L'index de la variance à inférer dans ABCsigm.
parsigm1 et parsigm2: Les hyperparamètres sur la variance sigma dans ABCsigm.
Npts: Le nombre de points souhaités dans la fonction Gibbs.

In [57]:
def GibbsABC(X, N, M, P, P2, Npts, objectifeps):
    start_time = time.time()  
    hyper = np.zeros((Npts, 3))
    hyper[0, :] = np.random.exponential(size=3)
    par1 = np.full((Npts, X.shape[1]), np.nan)
    par2 = np.full((Npts, X.shape[1]), np.nan)
    sigm = np.full((Npts, X.shape[1]), np.nan)
    hypersigm = np.full((Npts, 2), np.nan)
    hypersigm[0, :] = np.abs(cauchy.rvs(size=2))
    sigm[0, :] = 1 / gamma.rvs(hypersigm[0, 0], scale=hypersigm[0, 1], size=X.shape[1])
    
    simu_results = []  # Liste pour stocker les simulations
    distances_par = []  # Liste pour stocker les distances
    poids = []  # Liste pour stocker les poids
    histeps = []  # Liste pour stocker l'historique des epsilons
    cpu_times = []  # Liste pour stocker les temps CPU pour chaque distance calculée
    eps = float('inf')  # Initialiser epsilon à l'infini
    
    for i in range(1, Npts):
        dist_i = []
        poids_i = np.full(X.shape[1], np.nan)
        iteration_start_time = time.time()  # Enregistrer le temps de début de l'itération
        for j in range(X.shape[1]):
            U = ABCbas(X[:, j], N, sigm[i - 1, :], hyper[i - 1, :])
            par1[i, j] = U[0]
            par2[i, j] = U[1]
            sigm[i, j] = ABCsigm(X, j, par1[i, :], par2[i, :], hypersigm[i - 1, 0], hypersigm[i - 1, 1], P)
            simu_j = simu(par1[i, j], par2[i, j], sigm[i, j], X.shape[0])
            simu_results.append(simu_j)
            dist = np.abs(np.var(simu_j) - np.var(X[:, j]))
            dist_i.append(dist)
            poids_i[j] = 1 / (1 + dist)

        iteration_end_time = time.time()  # Enregistrer le temps de fin de l'itération
        cpu_time_for_iteration = iteration_end_time - iteration_start_time  # Calculer le temps CPU pour l'itération
        cpu_times.append(cpu_time_for_iteration)  # Ajouter le temps CPU à la liste

        eps = min(eps, max(dist_i))  # Mettre à jour epsilon avec la distance maximale de cette itération si elle est plus petite
        histeps.append(eps)  # Ajouter la valeur actuelle d'epsilon à l'historique
        if eps <= objectifeps:  # Vérifier si l'objectif est atteint
            print(f"Objectif epsilon atteint : {eps} <= {objectifeps}")
            break  # Sortir de la boucle si l'objectif est atteint

        hyper[i, :] = ABChaut(par1[i, :], par2[i, :], M)
        hypersigm[i, :] = ABCsigmhaut(sigm[i, :], P2)
        distances_par.append(dist_i)
        poids.append(poids_i / np.sum(poids_i))
        
    total_time = time.time() - start_time  # Calculer le temps total d'exécution
    
    resultats = {
        'Hyperparamètres': hyper,
        'Hypersigma': hypersigm,
        'Distances': distances_par,  # ou 'distances' selon la version de votre fonction
        'Historique des epsilons': histeps,
        'Temps CPU par itération': cpu_times,
        'Temps CPU total': total_time
    }
    
    # Pour imprimer les résultats avec des titres
    for titre, valeur in resultats.items():
        print(f"{titre}:")
        print(valeur)
        print("\n")  # Ajoute un espace entre chaque section pour la clarté

    return resultats  # Retourne le dictionnaire de résultats


In [51]:
l=50
X = np.random.normal(-5, 1, (20, l)) #data
N=1000 #n
P=100
P2=100
M=100
objectifeps=0.1
Npts=10


In [58]:
toyG = GibbsABC(X, N, M, P, P2, Npts, objectifeps)
toyG

C:\Users\sarah\AppData\Local\Temp\ipykernel_17772\3082693489.py:10: RuntimeWarning: divide by zero encountered in log
  return np.array([np.sum(np.log(q)), np.sum(np.log(np.abs(q - U[:, 0])))])
C:\Users\sarah\AppData\Local\Temp\ipykernel_17772\3082693489.py:15: RuntimeWarning: invalid value encountered in subtract
  dist = np.apply_along_axis(lambda x: np.sum((statdir(prior(len(Q1), x)) - statdir(Q))**2), 1, hyper)
C:\Users\sarah\AppData\Local\Temp\ipykernel_17772\3082693489.py:37: RuntimeWarning: divide by zero encountered in divide
  y = 1 / gamma.rvs(h[0], scale=h[1], size=n)


Hyperparamètres:
[[1.72884909 0.04045042 1.2324543 ]
 [0.76964402 0.07752307 0.01098683]
 [1.41272773 0.28985621 2.0552252 ]
 [0.43705435 0.22959494 0.29874472]
 [0.4259383  0.44109995 0.67443842]
 [0.48875937 0.63496349 1.33460846]
 [0.8061449  0.74468492 1.4999081 ]
 [0.60636189 1.07334546 0.74202667]
 [0.65010613 1.04185647 1.18224524]
 [0.32508871 0.80431988 0.15759686]]


Hypersigma:
[[ 0.42114119 10.58051802]
 [ 4.295319    0.34356698]
 [ 4.89336593  0.34962092]
 [ 3.1927656   0.40354316]
 [ 2.47061718  0.59034076]
 [ 5.41063408  0.2441797 ]
 [ 9.36586012  0.13727395]
 [ 3.98435479  0.30013193]
 [ 4.48426683  0.25807248]
 [ 1.8716028   0.67152845]]


Distances:
[[0.6041798118859267, 0.7728365088876792, 0.11829249876796588, 0.6529382726171516, 0.09753039837806876, 0.26990124023981976, 0.6966106426904032, 1.7787649009538948, 0.8891247974566903, 2.277654707270826, 7.216695160987188, 0.6962107590745966, 1.2156153939609577, 1.3274408838850638, 0.41069478457544834, 0.39226858099372697,

C:\Users\sarah\AppData\Local\Temp\ipykernel_17772\3082693489.py:37: RuntimeWarning: overflow encountered in divide
  y = 1 / gamma.rvs(h[0], scale=h[1], size=n)


{'Hyperparamètres': array([[1.72884909, 0.04045042, 1.2324543 ],
        [0.76964402, 0.07752307, 0.01098683],
        [1.41272773, 0.28985621, 2.0552252 ],
        [0.43705435, 0.22959494, 0.29874472],
        [0.4259383 , 0.44109995, 0.67443842],
        [0.48875937, 0.63496349, 1.33460846],
        [0.8061449 , 0.74468492, 1.4999081 ],
        [0.60636189, 1.07334546, 0.74202667],
        [0.65010613, 1.04185647, 1.18224524],
        [0.32508871, 0.80431988, 0.15759686]]),
 'Hypersigma': array([[ 0.42114119, 10.58051802],
        [ 4.295319  ,  0.34356698],
        [ 4.89336593,  0.34962092],
        [ 3.1927656 ,  0.40354316],
        [ 2.47061718,  0.59034076],
        [ 5.41063408,  0.2441797 ],
        [ 9.36586012,  0.13727395],
        [ 3.98435479,  0.30013193],
        [ 4.48426683,  0.25807248],
        [ 1.8716028 ,  0.67152845]]),
 'Distances': [[0.6041798118859267,
   0.7728365088876792,
   0.11829249876796588,
   0.6529382726171516,
   0.09753039837806876,
   0.26990124